In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
#from math import warnings
#warnings
#warnings.filterwarnings("ignore")

In [ ]:
import keras
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model,load_model
conv_base =  InceptionV3(weights='imagenet',include_top=False,
                         input_shape=(300, 300, 3))
output = conv_base.layers[-1].output
output = keras.layers.Flatten()(output)
model_tl = Model(conv_base.input, output)
model_tl.trainable = False
for layer in model_tl.layers:
    layer.trainable = False
layers = [(layer, layer.name, layer.trainable) for layer in  
               model_tl.layers]
model_layers=pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])
print(model_layers) 

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
test_size=400
batch_size=16
epochs=30
train_path='/content/drive/My Drive/NEWDATA/train'
test_path='/content/drive/My Drive/NEWDATA/val'
target_size=(300,300) 
train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.3,  
                                   rotation_range=50,
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2, 
                                   shear_range=0.2,
                                   horizontal_flip=True,
                                   brightness_range = [0.8, 1.2],
                                   fill_mode='nearest',        
                                   validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
                  train_path,
                  target_size=target_size,  
                  batch_size=batch_size,
                  class_mode='categorical',
                  subset='training')
validation_generator = train_datagen.flow_from_directory(
                       train_path,
                       target_size=target_size,
                       batch_size=batch_size,
                       class_mode='categorical',
                       subset='validation')

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
#from keras import optimizers
from tensorflow.keras import optimizers

model =Sequential()
model.add(model_tl)

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])
print(model.summary())

Model Checkpoint

In [ ]:
"""from keras.callbacks import *

filepath="/content/drive/My Drive/Models/epochs:{epoch:03d}-val_acc {val_acc:.3f}.hdf5"

checkpoint = ModelCheckpoint(filepath, 
                             monitor='val_acc', 
                             verbose=1,
                             save_best_only=False,
                             save_freq='epoch',     
                             mode='max')
callbacks_list = [checkpoint]"""

In [ ]:
history = model.fit(
          train_generator,
          steps_per_epoch=train_generator.samples//batch_size,    
          validation_data=validation_generator,
          validation_steps= validation_generator.samples//batch_size,
          epochs=epochs,
          verbose=1,
          shuffle=True
          )
#callbacks=callbacks_list

In [ ]:
# Model evaluation
scores_train = model.evaluate(train_generator,verbose=1)
scores_validation = model.evaluate(validation_generator,verbose=1)
print("Train Accuracy: %.2f%%" % (scores_train[1]*100))
print("Validation Accuracy: %.2f%%" % (scores_validation[1]*100))

def LearningCurve(history):
  plt.plot(history.history['acc'])
  plt.plot(history.history['val_acc'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'validation'], loc='upper left')
  plt.show()
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'validation'], loc='upper left')
  plt.show()
LearningCurve(history)
#Saving the trained model 
#model_weight_file='/content/drive/MyDrive/Models/TI_tlearn_img_aug_cnn_3.h5'
#model.save(model_weight_file)

In [ ]:
import math

In [ ]:
compute_steps_per_epoch = lambda x: int(math.ceil(1. * x / batch_size))
test_steps = compute_steps_per_epoch(test_size)
test_generator = test_datagen.flow_from_directory(
                 test_path,
                 target_size=target_size, 
                 batch_size=batch_size,
                 class_mode=None,
                 shuffle=False)
test_generator.reset()

#make predictions
#tl_img_aug_cnn = load_model(model_weight_file)
pred=model.predict(test_generator,
                            verbose=1,
                            steps=test_steps)
predicted_class_indices=np.argmax(pred,axis=1)
labels = (test_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})

import seaborn as sns
def PerformanceReports(conf_matrix,class_report,labels):
    ax= plt.subplot()
    sns.heatmap(conf_matrix, annot=True,ax=ax)
    ax.set_xlabel('Predicted labels')
    ax.set_ylabel('True labels')
    ax.set_title('Confusion Matrix')
    ax.xaxis.set_ticklabels(labels)
    ax.yaxis.set_ticklabels(labels)
    plt.show()
    ax= plt.subplot()
    sns.heatmap(pd.DataFrame(class_report).iloc[:-1, :].T,  
                annot=True,ax=ax)
    ax.set_title('Classification Report')
    plt.show()
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
labels=['original','tampered']
test_labels = [fn.split('/')[0] for fn in filenames]
cm=confusion_matrix(test_labels,predictions)
print(cm)
cr=classification_report(test_labels, predictions)
class_report=classification_report(test_labels, predictions,
                                   target_names=labels,
                                   output_dict=True)
print(cr)
PerformanceReports(cm,class_report,labels)

In [ ]:
import cv2
import os

In [ ]:
def sample_prediction(img_path):
    #test_im = cv2.resize(test_im, (img_size, img_size), cv2.INTER_LINEAR) / 255
    #test_pred = np.argmax(model.predict_proba(test_im.reshape((1, img_size, img_size, 3))))
    test_im = cv2.imread(img_path)
    test_im = cv2.resize(test_im, target_size, cv2.INTER_LINEAR)/255
    test_im = test_im.reshape((1, 300, 300, 3))
    test_pred = model.predict(test_im)
    test_pred = np.argmax(model(test_im))
    return test_pred

print("Predicted class for test_original: {}".format(sample_prediction("/content/drive/MyDrive/NEWDATA/train/Original/20210316_223932.jpg")))
print("Predicted class for test_tampered: {}".format(sample_prediction("/content/drive/MyDrive/NEWDATA/train/Tampered/174.jpg")))

In [ ]:
path =os.listdir("/content/drive/MyDrive/TI_Test/Tampered/")
counto = 0
countt = 0
total = 0
for file_name in path:
    test_im = os.path.join('/content/drive/MyDrive/TI_Test/Tampered/', file_name)
    test_im = cv2.imread(test_im)
    test_im = cv2.resize(test_im, target_size, cv2.INTER_LINEAR) / 255
    test_im = test_im.reshape((1, 300, 300, 3))
    #test_pred = np.argmax(model(test_im.reshape((1, 300, 300, 3))))
    test_pred = model.predict(test_im)
    #print(test_pred)
    #test_pred = np.argmax(model.predict(test_im))
    #print(test_pred)
    #print("")
    print("Filename: {} Predicted class: {}".format(file_name,test_pred))
    total+=1
    if test_pred[0][1] > 0.60: #test_pred[0][1]:
      countt+=1
    else:
      counto+=1

print("Total Count: {} Count Original: {}  Count Tampered: {} ".format(total,counto,countt))

In [ ]:
path =os.listdir("/content/drive/MyDrive/TI_Test/Original/")
counto = 0
countt = 0
total = 0
for file_name in path:
    test_im = os.path.join('/content/drive/MyDrive/TI_Test/Original/', file_name)
    test_im = cv2.imread(test_im)
    test_im = cv2.resize(test_im, target_size, cv2.INTER_LINEAR) / 255
    test_im = test_im.reshape((1, 300, 300, 3))
    #test_pred = np.argmax(model(test_im.reshape((1, 300, 300, 3))))
    test_pred = model.predict(test_im)
    #print(test_pred)
    #test_pred = np.argmax(model.predict(test_im))
    """print(test_pred)
    print("")
    print("")"""
    print("Filename: {} Predicted class: {}".format(file_name,test_pred))
    total+=1
    if test_pred[0][1] > 0.60: # test_pred[0][1]:
      countt+=1
    else:
      counto+=1

print("Total Count: {} Count Original: {}  Count Tampered: {} ".format(total,counto,countt))